In [1]:
import pandas as pd
import re
import quopri
from bs4 import BeautifulSoup
import numpy as np
import gdown

In [2]:
# https://drive.google.com/file/d/1cR-f4k6KElcFKr322aSeFF76qvg6-PeF/view?usp=sharing
gdown.download('https://drive.google.com/uc?id=1cR-f4k6KElcFKr322aSeFF76qvg6-PeF', '01_extracted_emails.csv', quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1cR-f4k6KElcFKr322aSeFF76qvg6-PeF
From (redirected): https://drive.google.com/uc?id=1cR-f4k6KElcFKr322aSeFF76qvg6-PeF&confirm=t&uuid=eb945676-fbef-4fd6-995b-36c0d497baba
To: /Users/pkchoy/code/data_science_bootcamp/predictor/notebooks/01_extracted_emails.csv
100%|██████████| 1.24G/1.24G [01:08<00:00, 17.9MB/s]


FileNotFoundError: [Errno 2] No such file or directory: '01_extracted_emails.csvl9i0djkc.part'

In [ ]:
# emails_df = pd.read_csv('../data/01_extracted_emails.csv')
emails_df = pd.read_csv('01_extracted_emails.csv')
print(emails_df.head(5))

                                    Message-ID  \
0  18782981.1075855378110.JavaMail.evans@thyme   
1  15464986.1075855378456.JavaMail.evans@thyme   
2  24216240.1075855687451.JavaMail.evans@thyme   
3  13505866.1075863688222.JavaMail.evans@thyme   
4  30922949.1075863688243.JavaMail.evans@thyme   

                                    Date                     From  \
0  Mon, 14 May 2001 16:39:00 -0700 (PDT)  phillip.allen@enron.com   
1   Fri, 4 May 2001 13:51:00 -0700 (PDT)  phillip.allen@enron.com   
2  Wed, 18 Oct 2000 03:00:00 -0700 (PDT)  phillip.allen@enron.com   
3  Mon, 23 Oct 2000 06:13:00 -0700 (PDT)  phillip.allen@enron.com   
4  Thu, 31 Aug 2000 05:07:00 -0700 (PDT)  phillip.allen@enron.com   

                        To    Subject   Cc  Mime-Version  \
0     tim.belden@enron.com        NaN  NaN           1.0   
1  john.lavorato@enron.com        Re:  NaN           1.0   
2   leah.arsdall@enron.com   Re: test  NaN           1.0   
3    randall.gay@enron.com        NaN  NaN  

In [ ]:
emails_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517401 entries, 0 to 517400
Data columns (total 18 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   Message-ID                 495554 non-null  object 
 1   Date                       495554 non-null  object 
 2   From                       495554 non-null  object 
 3   To                         495554 non-null  object 
 4   Subject                    478886 non-null  object 
 5   Cc                         124262 non-null  object 
 6   Mime-Version               495554 non-null  float64
 7   Content-Type               495554 non-null  object 
 8   Content-Transfer-Encoding  495554 non-null  object 
 9   Bcc                        126416 non-null  object 
 10  X-From                     495554 non-null  object 
 11  X-To                       495554 non-null  object 
 12  X-cc                       127172 non-null  object 
 13  X-bcc                      16

In [ ]:
emails_df.isnull().sum()

Message-ID                    21847
Date                          21847
From                          21847
To                            21847
Subject                       38515
Cc                           393139
Mime-Version                  21847
Content-Type                  21847
Content-Transfer-Encoding     21847
Bcc                          390985
X-From                        21847
X-To                          21847
X-cc                         390229
X-bcc                        517233
X-Folder                      21847
X-Origin                      21847
X-FileName                    22394
Message-Body                  21848
dtype: int64

In [ ]:
emails_df.dropna(how='all', inplace=True)
emails_df.dropna(subset=['Message-Body'], inplace=True)

In [ ]:
# drop columns with lots of missing values
print(emails_df['Bcc'].isnull().mean() * 100, '% of X-bcc is empty')
print(emails_df['Cc'].isnull().mean() * 100, '% of X-cc is empty')
print(emails_df['X-bcc'].isnull().mean() * 100, '% of X-bcc is empty')
print(emails_df['X-cc'].isnull().mean() * 100, '% of X-cc is empty')

emails_df.drop(columns=['Bcc', 'Cc', 'X-bcc', 'X-cc'], inplace=True)
print('remaining cols: ', emails_df.columns)

74.4899132887905 % of X-bcc is empty
74.92457920747125 % of X-cc is empty
99.96609847988005 % of X-bcc is empty
74.33735644825074 % of X-cc is empty
remaining cols:  Index(['Message-ID', 'Date', 'From', 'To', 'Subject', 'Mime-Version',
       'Content-Type', 'Content-Transfer-Encoding', 'X-From', 'X-To',
       'X-Folder', 'X-Origin', 'X-FileName', 'Message-Body'],
      dtype='object')


In [ ]:
# Impute with No Subject
emails_df['Subject'].fillna('No Subject', inplace=True)

/var/folders/cv/swyglqyx1tlc_6vc0p615fhr0000gn/T/ipykernel_70781/1007409803.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  emails_df['Subject'].fillna('No Subject', inplace=True)


In [ ]:
# Drop columns that are not useful for prediction
emails_df = emails_df.drop(columns=['Message-ID', 'Mime-Version', 'Content-Type', 'Content-Transfer-Encoding', 'X-Folder', 'X-Origin', 'X-FileName'])

In [ ]:
emails_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 495553 entries, 0 to 517400
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   Date          495553 non-null  object
 1   From          495553 non-null  object
 2   To            495553 non-null  object
 3   Subject       495553 non-null  object
 4   X-From        495553 non-null  object
 5   X-To          495553 non-null  object
 6   Message-Body  495553 non-null  object
dtypes: object(7)
memory usage: 30.2+ MB


In [ ]:
print(emails_df['Subject'][9], emails_df['Message-Body'][9])

FW: fixed forward or other Collar floor gas price terms ---------------------- Forwarded by Phillip K Allen/HOU/ECT on 10/16/2000 
01:42 PM ---------------------------


"Buckner, Buck" <buck.buckner@honeywell.com> on 10/12/2000 01:12:21 PM
To: "'Pallen@Enron.com'" <Pallen@Enron.com>
cc:  
Subject: FW: fixed forward or other Collar floor gas price terms


Phillip,

> As discussed  during our phone conversation, In a Parallon 75 microturbine
> power generation deal for a national accounts customer, I am developing a
> proposal to sell power to customer at fixed or collar/floor price. To do
> so I need a corresponding term gas price for same. Microturbine is an
> onsite generation product developed by Honeywell to generate electricity
> on customer site (degen). using natural gas. In doing so,  I need your
> best fixed price forward gas price deal for 1, 3, 5, 7 and 10 years for
> annual/seasonal supply to microturbines to generate fixed kWh for
> customer. We have the opportunity to sel

In [ ]:
emails_df['Message-Body']

0                                 Here is our forecast\n\n 
1         Traveling to have a business meeting takes the...
2                            test successful.  way to go!!!
3         Randy,\n\n Can you send me a schedule of the s...
4                       Let's shoot for Tuesday at 11:45.  
                                ...                        
517396    This is a trade with OIL-SPEC-HEDGE-NG (John L...
517397    Some of my position is with the Alberta Term b...
517398    2\n\n -----Original Message-----\nFrom: \tDouc...
517399    Analyst\t\t\t\t\tRank\n\nStephane Brodeur\t\t\...
517400    i think the YMCA has a class that is for peopl...
Name: Message-Body, Length: 495553, dtype: object

In [ ]:
# Find the emails that have common reply or forward patterns
# -+\s*(Original|Forwarded)
def find_reply_forward(msg_body):

    if not isinstance(msg_body, str):
        return False

    pattern = re.compile(
        r"""
        (?:-+\s*(?:Original|Forwarded)) # forwarded
        |On\s+[^\n]+wrote: # replies
        |>\s*(?:From|Sent|To): # Quoted replies
        |Note:\s*forwarded\s*message\s*attached  # Forwarded note
        |Begin\s+forwarded\s+message
        |Message\s+forwarded
        """,
        re.IGNORECASE | re.VERBOSE | re.DOTALL)
    return bool(pattern.search(msg_body))

# emails_df['has_reply_forward_in_msg'] = emails_df['Message-Body'].apply(find_reply_forward)

print('before: ', emails_df.shape[0])
emails_df = emails_df[~emails_df['Message-Body'].apply(find_reply_forward)]
print('after: ', emails_df.shape[0])


before:  495553
after:  320509


In [ ]:
def find_forward_subject(subject):
    if not isinstance(subject, str):
        return False

    pattern = re.compile(r'^\s*(?:FW:|Fwd:)', re.IGNORECASE)
    return bool(pattern.search(subject))

print('before: ', emails_df.shape[0])
emails_df = emails_df[~emails_df['Subject'].apply(find_forward_subject)]
print('after: ', emails_df.shape[0])

before:  320509
after:  317504


In [ ]:
# emails_df['has_reply_forward_in_msg'].value_counts()

In [ ]:
# Remove the emails that have reply or forward
# emails_df = emails_df[emails_df['has_reply_forward_in_msg'] == False]
# emails_df['has_reply_forward_in_msg'].value_counts()
# emails_df.drop(columns=['has_reply_forward_in_msg'], inplace=True)


In [ ]:
# clean msg body
email_endings = ['best regards', 'regards', 'best', 'All the best', 'sincerely', 'cheers', 'thanks',
                'thank you', 'warm regards', 'yours truly', 'yours sincerely', 'yours faithfully',
                'yours', 'kind regards', 'cordially', 'respectfully', 'with regards', 'with best regards',
                  'with warm regards', 'with sincere regards', 'with gratitude', 'with appreciation', 'with thanks',
                    'with warmest regards', 'with deepest gratitude', 'with deepest appreciation', 'with deepest thanks']
escaped_endings = [re.escape(endings) for endings in email_endings]

common_signature = ['Enron North America Corp.', 'Carol St. Clair', 'Debra Perlingiere', 'Keegan Farrell', 'Eric Bass']
escaped_signature = [re.escape(sig) for sig in common_signature]


IMAGE_PATTERN = re.compile(r'\[IMAGE\]')
URL_PATTERN = re.compile(r'https?://\S+|www\.\S+')
FILE_PATTERN = re.compile(r'[-|\s]?[a-zA-Z0-9_-]+\.(?:txt|pdf|docx?|xlsx?|pptx?|csv)')
PHONE_NUM_START_PATTERN = re.compile(r'(?i)^\s*(?:\(?(phone|fax|mobile|mob|tel|office)\)?:?\s*)?(?:\+?\d{1,3}[\s\-]?)?\d{2,4}[\s\-]?\d{3}[\s\-]?\d{4}[\s\S]*$')
EMAIL_SIGNOFF_PATTERN = re.compile(r'(?im)^\s*(?:' + "|".join(escaped_endings) + r')[\s\S]*')
EMAIL_SIGNATURE_PATTERN = re.compile(r'(?im)^\s*(?:' + "|".join(escaped_signature) + r')[\s\S]*')
EMAIL_HEADER_PATTERN = re.compile(r'(?:Subject|To|Cc|Bcc|From|Sent|Content-Type|Content-Transfer-Encoding|MIME-Version):\s+.*\n', re.IGNORECASE)
PRICE_MASK_PATTERN = re.compile(r'\$\d+(?:\.\d{1,2})?(?:\s*[-\u2013*]\s*\$\d+(?:\.\d{1,2})?)?')
PERCENTAGE_MASK_PATTERN = re.compile(r'(?:\d+(?:\.\d{1,3})?(?:\s*\%(?:\s*[-\u2013*]\s*\d+(?:\.\d{1,3})?\s*\%?)?))')
PHONE_PATTERN = re.compile(r'(?i)(?:\+?\d{1,3}[\s\-\.]?)?\(?\d{2,4}\)?[\s\-\.]?\d{2,3}[\s\-\.]?\d{3,4}')
LOCAL_PHONE_PATTERN = re.compile(r'\b\d{3}[-\s\.]?\d{4}\b')
EMAIL_PATTERN = re.compile(r'\b[\w\.-]+@[\w\.-]+\.\w+\b')
NUM_DATE_PATTERN = re.compile(r'\b(?:\d{1,2}[-/]\d{1,2}[-/]\d{2,4}|\d{2,4}[-/]\d{1,2}[-/]\d{1,2})\b')
TEXTUAL_DATE_PATTERN = re.compile(r'(?i)\b(?:(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Sept|Oct|Nov|Dec|January|February|March|April|May|June|July|August|September|October|November|December)\s+\d{1,2}(?:st|nd|rd|th)?(?:,)?\s+\d{4})\b')
TIME_PATTERN = re.compile(r'(?i)\b\d{1,2}:\d{2}(:\d{2})?\s*(AM|PM)?\b')
RECIPIENT_BLOCK_PATTERN = re.compile(r'(?i)\n\s*[A-Z][a-z]+(?:\s+[A-Z][a-z]+)?/[A-Z]+@[\w/]+[\s\S]*',)
TAB_SPACE_PATTERN = re.compile(r'[ \t]+')
MULTIPLE_NEWLINES_PATTERN = re.compile(r'\n{2,}')
# PROMO_PATTERN = re.compile(r"""
#                         unsubscribe|limited time offer|buy now|discount|promo code|click here|order now|shop now
#                         |save now|exclusive deal|offer expires|last chance|order today|shop today|limited offer
#                         |special offer|guarantee"""
#                         ,re.IGNORECASE | re.VERBOSE)

def clean_html(msg_body):
    return BeautifulSoup(msg_body, 'html.parser').get_text()

def clean_msg_body(msg_body):
    if not isinstance(msg_body, str):
        return ''

    # Try decoding Quoted-Printable format
    try:
        msg_body = quopri.decodestring(msg_body).decode('utf-8', errors='ignore')
    except Exception:
        pass   # If decoding fails, keep the original message

    msg_body = clean_html(msg_body)
    msg_body = EMAIL_SIGNOFF_PATTERN.sub('', msg_body)
    msg_body = EMAIL_SIGNATURE_PATTERN.sub('', msg_body)
    msg_body = PHONE_NUM_START_PATTERN.sub('', msg_body)
    msg_body = EMAIL_HEADER_PATTERN.sub('', msg_body)
    msg_body = IMAGE_PATTERN.sub('', msg_body)
    msg_body = URL_PATTERN.sub('', msg_body)
    msg_body = FILE_PATTERN.sub('', msg_body)
    msg_body = RECIPIENT_BLOCK_PATTERN.sub('', msg_body)
    msg_body = PRICE_MASK_PATTERN.sub('$X', msg_body)
    msg_body = EMAIL_PATTERN.sub('[EMAIL]', msg_body)
    msg_body = PERCENTAGE_MASK_PATTERN.sub('X%', msg_body)
    msg_body = PHONE_PATTERN.sub('[PHONE]', msg_body)
    msg_body = LOCAL_PHONE_PATTERN.sub('[PHONE]', msg_body)
    msg_body = NUM_DATE_PATTERN.sub('[DATE]', msg_body)
    msg_body = TEXTUAL_DATE_PATTERN.sub('[DATE]', msg_body)
    msg_body = TIME_PATTERN.sub('[TIME]', msg_body)
    msg_body = TAB_SPACE_PATTERN.sub(' ', msg_body)

    # Collapse multiple newlines, but preserve one newline between blocks of text
    msg_body = MULTIPLE_NEWLINES_PATTERN.sub('\n\n', msg_body)

    # Strip each line and remove lines that are purely whitespace
    lines = [line.strip() for line in msg_body.split('\n')]
    msg_body = '\n'.join([line for line in lines if line])

    msg_body = msg_body.strip()

    return msg_body

emails_df['Message-Body'] = emails_df['Message-Body'].apply(clean_msg_body)

/var/folders/cv/swyglqyx1tlc_6vc0p615fhr0000gn/T/ipykernel_70781/871204959.py:38: MarkupResemblesLocatorWarning: The input passed in on this line looks more like a URL than HTML or XML.

If you meant to use Beautiful Soup to parse the web page found at a certain URL, then something has gone wrong. You should use an Python package like 'requests' to fetch the content behind the URL. Once you have the content as a string, you can feed that string into Beautiful Soup.

However, if you want to parse some data that happens to look like a URL, then nothing has gone wrong: you are using Beautiful Soup correctly, and this warning is spurious and can be filtered. To make this warning go away, run this code before calling the BeautifulSoup constructor:

    from bs4 import MarkupResemblesLocatorWarning
    import warnings

    warnings.filterwarnings("ignore", category=MarkupResemblesLocatorWarning)
    
  return BeautifulSoup(msg_body, 'html.parser').get_text()
/var/folders/cv/swyglqyx1tlc_6vc0

In [ ]:
emails_df.describe().T

,count,unique,top,freq
Date,317504,136816,"Mon, 31 Dec 1979 16:00:00 -0800 (PST)",281
From,317504,17468,pete.davis@enron.com,9147
To,317504,39191,pete.davis@enron.com,9146
Subject,317504,101570,No Subject,14810
X-From,317504,23377,Enron Announcements,8525
X-To,317504,47783,pete.davis@enron.com,5334
Message-Body,317504,133025,,8845


In [ ]:
# drop duplicates
print(emails_df['Message-Body'].value_counts())
emails_df.drop_duplicates(subset=['Message-Body'], inplace=True)
emails_df.dropna(subset=['Message-Body'], inplace=True)
emails_df.reset_index(drop=True, inplace=True)

Message-Body
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            8845
LOG MESSAGES:\nPARSING FILE -->> O:\Portland\WestDesk\California Scheduling\ISO Final Schedules\                                                                                                                                                                                                                                                                                                                                                                                          

In [ ]:
emails_df.describe().T

,count,unique,top,freq
Date,133025,121785,"Mon, 31 Dec 1979 16:00:00 -0800 (PST)",167
From,133025,16762,jeff.dasovich@enron.com,2746
To,133025,37397,sara.shackleton@enron.com,1485
Subject,133025,93765,No Subject,6265
X-From,133025,22128,Jeff Dasovich,2308
X-To,133025,44904,klay@enron.com,1240
Message-Body,133025,133025,Here is our forecast,1


In [ ]:
# Remove short, low value emails
def is_low_value(msg_body, min_word_count=25):
    if not isinstance(msg_body, str):
        return False
    # if len(msg_body.split()) < min_word_count:

    msg_cleaned = re.sub(r'[^a-zA-Z]', ' ', msg_body).strip()
    return len(msg_cleaned.split()) < min_word_count

print('before: ', emails_df.shape[0])
emails_df = emails_df[~emails_df['Message-Body'].apply(is_low_value)]
print('after: ', emails_df.shape[0])

before:  133025
after:  100726


In [ ]:
MACHINE_GEN_REPORT_PATTERN = re.compile(r'^The\sreport\snamed:.*\n*(?:.*\n)*\(Revision: \d+\)\s*$')
SUBSCRIPTION_PATTERN = re.compile(r'(?i)unsubscribe|subscription|click here to unsubscribe|click here to manage your subscription|click here to change your subscription|click here to update your subscription|click here to change your email preferences|click here to unsubscribe|click here to manage your email preferences|click here to update')

def is_unwanted(msg_body):
    if not isinstance(msg_body, str):
        return False
    if MACHINE_GEN_REPORT_PATTERN.match(msg_body):
        return True
    if SUBSCRIPTION_PATTERN.search(msg_body):
        return True
    return False

print('before: ', emails_df.shape[0])
emails_df = emails_df[~emails_df['Message-Body'].apply(is_unwanted)]
print('after: ', emails_df.shape[0])

before:  100726
after:  92376


In [ ]:
# Remove emails that are auto-generated or system generated
NOREPLY_PATTERN = re.compile(r'(?i)no(\.|-)?reply|do(\.|-)?not(\.|-)?reply|auto(\.|-)?generated|system|notification|daemon')

def is_bad_source(sender):
    if not isinstance(sender, str):
        return False

    return bool(re.search(NOREPLY_PATTERN, sender))

print('before: ', emails_df.shape[0])
emails_df = emails_df[~emails_df['From'].apply(is_bad_source)]
print('after: ', emails_df.shape[0])

before:  92376
after:  90780


In [ ]:
# Remove emails that are auto-generated or system generated
STRUCTURED_REPORT_PATTERN = ['newsletter', 'weekly report', 'daily report', 'bulletin', 'digest', 'notification', 'weekly digest', 'daily digest', 'monthly digest', 'quarterly digest', 'annual digest']

def is_unwanted_subject(subject):
    if not isinstance(subject, str):
        return False
    if any(keyword in subject for keyword in STRUCTURED_REPORT_PATTERN):
        return True
    return False

print('before: ', emails_df.shape[0])
emails_df = emails_df[~emails_df['Subject'].apply(is_unwanted_subject)]
print('after: ', emails_df.shape[0])

before:  90780
after:  90747


In [ ]:
# Remove emails from non-enron sources
def is_from_enron(sender):
    if not isinstance(sender, str):
        return False

    return sender.lower().endswith('enron.com')

print('before: ', emails_df.shape[0])
emails_df = emails_df[emails_df['From'].apply(is_from_enron)]
print('after: ', emails_df.shape[0])

before:  90747
after:  66276


In [ ]:
self_sent_count = (emails_df["From"] == emails_df["To"]).sum()
print(self_sent_count)

1813


In [ ]:
print(emails_df[emails_df["From"] == emails_df["To"]].sample(10))

                                        Date                    From  \
25680  Mon, 15 Oct 2001 15:38:27 -0700 (PDT)    pete.davis@enron.com   
21868  Wed, 27 Dec 2000 14:06:00 -0800 (PST)   steven.kean@enron.com   
40649  Mon, 16 Apr 2001 14:43:00 -0700 (PDT)    pete.davis@enron.com   
40642  Sun, 29 Apr 2001 23:41:00 -0700 (PDT)    pete.davis@enron.com   
40616  Wed, 18 Apr 2001 08:42:00 -0700 (PDT)    pete.davis@enron.com   
40992  Thu, 26 Apr 2001 11:41:00 -0700 (PDT)    pete.davis@enron.com   
40985   Tue, 6 Mar 2001 23:39:00 -0800 (PST)    pete.davis@enron.com   
25800  Sun, 21 Oct 2001 19:49:31 -0700 (PDT)    pete.davis@enron.com   
39974  Wed, 28 Feb 2001 07:38:00 -0800 (PST)    pete.davis@enron.com   
88735  Tue, 20 Nov 2001 17:22:57 -0800 (PST)  gerald.nemec@enron.com   

                           To  \
25680    pete.davis@enron.com   
21868   steven.kean@enron.com   
40649    pete.davis@enron.com   
40642    pete.davis@enron.com   
40616    pete.davis@enron.com   
40992    

In [ ]:
# Remove self-sent emails, coz usually are automated logs or reports
print('before: ', emails_df.shape[0])
emails_df = emails_df[emails_df['From'] != emails_df['To']]
print('after: ', emails_df.shape[0])

before:  66276
after:  64463


In [ ]:
print('before: ', emails_df.columns)
emails_df = emails_df.rename(columns={'Message-Body': 'Message', 'X-From': 'X_From', 'X-To': 'X_To'})
print('after: ', emails_df.columns)
emails_df.reset_index(drop=True, inplace=True)

before:  Index(['Date', 'From', 'To', 'Subject', 'X-From', 'X-To', 'Message-Body'], dtype='object')
after:  Index(['Date', 'From', 'To', 'Subject', 'X_From', 'X_To', 'Message'], dtype='object')


In [ ]:
EMAIL_PATTERN = re.compile(r'\b[\w\.-]+@[\w\.-]+\.\w+\b')

def clean_emails_in_subject(subject):
    if not isinstance(subject, str):
        return ''

    return EMAIL_PATTERN.sub('[EMAIL]', subject)

emails_df['Subject'] = emails_df['Subject'].apply(clean_emails_in_subject)

In [ ]:
emails_df['Entire-Message'] = emails_df['Subject'].fillna('') + ' ' + emails_df['Message'].fillna('')

In [ ]:
emails_df['DISC_rule'] = pd.NA
emails_df['DISC_manual'] = pd.NA
emails_df['DISC_final'] = pd.NA
emails_df.isnull().sum()

Date                  0
From                  0
To                    0
Subject               0
X_From                0
X_To                  0
Message               0
Entire-Message        0
DISC_rule         64463
DISC_manual       64463
DISC_final        64463
dtype: int64

In [ ]:
# emails_df = emails_df.drop(columns=['DISC'])

In [ ]:
emails_df.to_csv('../data/02_cleaned_emails_unlabelled_internal.csv', index=False)

In [ ]:
# emails_df.to_csv('../data/02_cleaned_emails_unlabelled_below_100Mb.csv', index=False)

In [ ]:
emails_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64463 entries, 0 to 64462
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Date            64463 non-null  object
 1   From            64463 non-null  object
 2   To              64463 non-null  object
 3   Subject         64463 non-null  object
 4   X_From          64463 non-null  object
 5   X_To            64463 non-null  object
 6   Message         64463 non-null  object
 7   Entire-Message  64463 non-null  object
 8   DISC_rule       0 non-null      object
 9   DISC_manual     0 non-null      object
 10  DISC_final      0 non-null      object
dtypes: object(11)
memory usage: 5.4+ MB


In [ ]:
emails_df.head(5)

,Date,From,To,Subject,X_From,X_To,Message,Entire-Message,DISC_rule,DISC_manual,DISC_final
0,"Fri, 4 May 2001 13:51:00 -0700 (PDT)",phillip.allen@enron.com,john.lavorato@enron.com,Re:,Phillip K Allen,John J Lavorato <John J Lavorato/ENRON@enronXg...,Traveling to have a business meeting takes the...,Re: Traveling to have a business meeting takes...,<NA>,<NA>,<NA>
1,"Mon, 23 Oct 2000 06:13:00 -0700 (PDT)",phillip.allen@enron.com,randall.gay@enron.com,No Subject,Phillip K Allen,Randall L Gay,"Randy,\nCan you send me a schedule of the sala...","No Subject Randy,\nCan you send me a schedule ...",<NA>,<NA>,<NA>
2,"Tue, 17 Oct 2000 02:26:00 -0700 (PDT)",phillip.allen@enron.com,mark.scott@enron.com,Re: High Speed Internet Access,Phillip K Allen,Mark Scott,1. login: pallen pw: ke9davis\nI don't think t...,Re: High Speed Internet Access 1. login: palle...,<NA>,<NA>,<NA>
3,"Mon, 16 Oct 2000 06:42:00 -0700 (PDT)",phillip.allen@enron.com,buck.buckner@honeywell.com,Re: FW: fixed forward or other Collar floor ga...,Phillip K Allen,"""Buckner, Buck"" <buck.buckner@honeywell.com> @...","Mr. Buckner,\nFor delivered gas behind San Die...",Re: FW: fixed forward or other Collar floor ga...,<NA>,<NA>,<NA>
4,"Fri, 13 Oct 2000 06:45:00 -0700 (PDT)",phillip.allen@enron.com,stagecoachmama@hotmail.com,No Subject,Phillip K Allen,stagecoachmama@hotmail.com,"Lucy,\nHere are the rentrolls:\nOpen them and ...","No Subject Lucy,\nHere are the rentrolls:\nOpe...",<NA>,<NA>,<NA>


In [ ]:
# email_100 = emails_df['Message'].sample(100)
# print(email_100)


In [ ]:
# email_100.to_csv('../data/sample_100.csv', index=False)